In [112]:
import hashlib

In [113]:
def sha256(m):
    return int(hashlib.sha256(m).hexdigest(),16)

In [114]:
p = 2**256 - 2**224 + 2**192 + 2**96 - 1
q = 0xffffffff00000000ffffffffffffffffbce6faada7179e84f3b9cac2fc632551
xG = 0x6b17d1f2e12c4247f8bce6e563a440f277037d812deb33a0f4a13945d898c296
yG = 0x4fe342e2fe1a7f9b8ee7eb4a7c0f9e162bce33576b315ececbb6406837bf51f5
a = p - 3

In [115]:
def shift(x, i):
    return (x >> i) & 1

def gf_mul(a, b):
    return (a*b) % p

def gf_add(a, b):
    return (a+b) % p

def gf_sub(a, b):
    return (a-b) % p

def gf_exp(z, e, mod=p):
    x = z
    for i in range(254, -1, -1):
        x = (x*x) % mod
        if shift(e, i):
            x = (x*z) % mod
    return x

def ec_add(x1, y1, x2, y2):
    if x1 == 0 and y1 == 0:
        return x2, y2
    if x2 == 0 and y2 == 0:
        return x1, y1

    r0 = (y2 - y1) % p
    r1 = (x2 - x1) % p
    lmbd = (r0 * gf_exp(r1, p-2)) % p
    x3 = (lmbd * lmbd - x1 - x2) % p
    y3 = (lmbd * (x1 - x3) - y1) % p
    return x3, y3

def ec_dub(x1, y1):
    if x1 == 0 and y1 == 0:
        return 0, 0
    r0 = (3*x1*x1 + a) % p
    r1 = (2*y1) % p
    lmbd = (r0 * gf_exp(r1, p-2)) % p
    x3 = (lmbd * lmbd - x1 - x1) % p
    y3 = (lmbd * (x1 - x3) - y1) % p
    return x3, y3



In [125]:
def ec_spm(k, x, y):
    xQ = 0
    yQ = 0

    for i in range(255, -1, -1):
        xQ, yQ = ec_dub(xQ, yQ)
        if shift(k, i):
            xQ, yQ = ec_add(xQ, yQ, xG, yG)
    
    return xQ, yQ

def ec_sign(k, d, m):
    x, y = ec_spm(k, xG, yG)
    r = x % q
    e = sha256(m)
    print("e : ", hex(e))
    invk = gf_exp(k, q-2, q)
    print("invk : ", hex(invk))
    s = (invk * (e + d*r)) % q
    return r, s


In [126]:
m = 0x5905238877c77421f73e43ee3da6f2d9e2ccad5fc942dcec0cbd25482935faaf416983fe165b1a045ee2bcd2e6dca3bdf46c4310a7461f9a37960ca672d3feb5473e253605fb1ddfd28065b53cb5858a8ad28175bf9bd386a5e471ea7a65c17cc934a9d791e91491eb3754d03799790fe2d308d16146d5c9b0d0debd97d79ce8

test_d = 0x519b423d715f8b581f4fa8ee59f4771a5b44c8130b4e3eacca54a56dda72b464
test_Qx = 0x1ccbe91c075fc7f4f033bfa248db8fccd3565de94bbfb12f3c59ff46c271bf83
test_Qy = 0xce4014c68811f9a21a1fdb2c0e6113e06db7ca93b7404e78dc7ccd5ca89a4ca9
test_k = 0x94a1bbb14b906a61a280f245f9e93c7f3b4a6247824f5d33b9670787642a68de
test_R = 0xf3ac8061b514795b8843e3d6629527ed2afd6b1f6a555a7acabb5e6f79c8c2ac
test_S = 0x8bf77819ca05a6b2786c76262bf7371cef97b218e96f175a3ccdda2acc058903

r, s = ec_sign(test_k, test_d, m.to_bytes(128, byteorder='big'))
print("r : ", hex(r))
print("s : ", hex(s))
print(r == test_R and s == test_S)


e :  0x44acf6b7e36c1342c2c5897204fe09504e1e2efb1a900377dbc4e7a6a133ec56
invk :  0x3b549196a13c898a6f6e84dfb3a22c40a8b9b17fb88e408ea674e451cd01d0a6
r :  0xf3ac8061b514795b8843e3d6629527ed2afd6b1f6a555a7acabb5e6f79c8c2ac
s :  0x8bf77819ca05a6b2786c76262bf7371cef97b218e96f175a3ccdda2acc058903
True
